In [1]:
from ray.rllib.agents import ppo, dqn
from ray import tune
import numpy as np
import torch

import sys
sys.path.append("../..")
from thesis.utils.utils import get_config, setup_ray, save, load
setup_ray()

In [2]:
env_args = dict(
    fleetsize = 2,
    max_fleetsize = 30,    
    pseudo_dispatcher = True,
    routing_agent_death= True,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 1, 
        reward_wrong_target = -0.3,
        reward_removed_for_block = -3, 
        reward_target_distance = 0.2,
        reward_invalid=-0.1,
        block_timeout = 20,
        reward_accepted_in_station = 1,
        reward_declined_in_station = -1,
        dispatching_interval=240,
        io_quote = 0.99  ,
        availability = 0.95,
        mttr = 5*60,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "lin_model",
        custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=32,
            with_action_mask=True,
            with_agvs=True,
            with_stations = False,
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = True,
        )
    )
)

In [4]:
config, logger_creator, checkpoint_dir = get_config(
    batch_size=5000,
    env_args = env_args, 
    agv_model = agv_model,
    train_agv = True,
    dispatcher_model=dispatcher_model, 
    train_dispatcher=True,
    env = "matrix"
)
trainer = ppo.PPOTrainer(config, logger_creator=logger_creator)

2022-11-07 21:15:29,916	WARNING ppo.py:386 -- `train_batch_size` (5000) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=4 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1250.
2022-11-07 21:15:29,917	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-11-07 21:15:29,920	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_88576_98km8hr9)
  warn(f"Unzipping to temporary directory ({tmp_dir})")


In [ ]:
#trainer.restore("../../models/Default/1_10_2022-11-07_17-38-06/checkpoint_000020/checkpoint-20")

In [ ]:
for j in range(500):
    for i in range(20):
        trainer.train()    
    trainer.save(checkpoint_dir)

ModelError: ModelError[path=, status={'reason': 'MODEL_ERROR Caused by class java.lang.ArrayIndexOutOfBoundsException: Index 24 out of bounds for length 24', 'next_event_time': 480.1, 'episode_count': 4, 'step_count': 435, 'model_time': 480.1}, error=None, msg='']